In [1]:
import os
import pandas as pd
import numpy as np
import tqdm
import pydicom
import glob
import os.path
import shutil

# pd.set_option('display.width', 1000)
# pd.set_option('display.max_rows', 1000)
# pd.set_option('display.max_columns', 1000)

In [2]:
input_dir= "input_data/UP2127/Up2127_Us_2/Abd_Peds - 20220531112241/unnamed_1/"
sorted_dir='sorted_data/'
human_dir = 'human_check/'

if not os.path.exists(human_dir):
    os.mkdir(human_dir)
    
if not os.path.exists(sorted_dir):
        os.mkdir(sorted_dir)
        
def human_check(in_dir= input_dir):
    file_names = os.listdir(in_dir)
    for file_name in file_names:
        shutil.move(os.path.join(input_dir, file_name), human_dir)

def empty_dirs_check(sorted_dir):
    empty_dirs=[]
    for root, _, files in os.walk(sorted_dir):
        if os.listdir(root) == []:
            empty_dirs.append(root)
    return empty_dirs

def delete_ini_ds_store(in_dir):
    for root, _, files in os.walk(in_dir):
        ini_file = root + '/desktop.ini'
        print(ini_file)
        if os.path.exists(ini_file):
            os.remove(ini_file)
        ds_store_file = root + '/.DS_Store'   
        print(ds_store_file)
        if os.path.exists(ds_store_file):
            os.remove(ds_store_file)

In [3]:
def save_value(img_data, name, value):
    if type(value) == pydicom.multival.MultiValue:
        for i, j in enumerate(value):
            save_value(img_data, f"{name}_{i}", j)
    else:
        if type(value) == pydicom.uid.UID:
            value = str(value)
        elif type(value) == pydicom.valuerep.DSfloat:
            value = float(value)
        img_data[name] = value

def get_data_dict(img):
    img_data = {}
    for i in img.iterall():
        if i.name == "Pixel Data":
            continue
        name = i.name.replace(" ", "_").replace("(", "").replace(")", "").lower()
        save_value(img_data, name, i.value)
    return img_data

def get_list_data(imgs):
    list_data = []
    for i in tqdm.tqdm(imgs):
        img = pydicom.read_file(i)
        img_data = get_data_dict(img)
#         print(i)
        list_data.append(img_data)
    return list_data

def get_df_data(imgs):
    list_data = get_list_data(imgs)
    return pd.DataFrame(list_data)

In [4]:
delete_ini_ds_store(input_dir)
delete_ini_ds_store(sorted_dir)
delete_ini_ds_store(human_dir)

input_data/UP2127/Up2127_Us_2/Abd_Peds - 20220531112241/unnamed_1//desktop.ini
input_data/UP2127/Up2127_Us_2/Abd_Peds - 20220531112241/unnamed_1//.DS_Store
sorted_data//desktop.ini
sorted_data//.DS_Store
sorted_data/UP2127/desktop.ini
sorted_data/UP2127/.DS_Store
sorted_data/UP2127\Exam1/desktop.ini
sorted_data/UP2127\Exam1/.DS_Store
sorted_data/UP2127\Exam1\Clinical_Abd_Wall/desktop.ini
sorted_data/UP2127\Exam1\Clinical_Abd_Wall/.DS_Store
sorted_data/UP2127\Exam1\Clinical_Cine/desktop.ini
sorted_data/UP2127\Exam1\Clinical_Cine/.DS_Store
sorted_data/UP2127\Exam1\Clinical_Static/desktop.ini
sorted_data/UP2127\Exam1\Clinical_Static/.DS_Store
sorted_data/UP2127\Exam1\RF/desktop.ini
sorted_data/UP2127\Exam1\RF/.DS_Store
sorted_data/UP2127\Exam1\RF\HarmonicsOFF/desktop.ini
sorted_data/UP2127\Exam1\RF\HarmonicsOFF/.DS_Store
sorted_data/UP2127\Exam1\RF\HarmonicsOFF\Liver/desktop.ini
sorted_data/UP2127\Exam1\RF\HarmonicsOFF\Liver/.DS_Store
sorted_data/UP2127\Exam1\RF\HarmonicsOFF\PhantomContem

In [5]:
delete_ini_ds_store(human_dir)

human_check//desktop.ini
human_check//.DS_Store


In [6]:
# move_report_files(glob.glob(input_dir + '*.dcm'))
files_path_list = []
report_list = []
for filename in glob.glob(input_dir + '*.dcm'):
    if os.path.getsize(filename) <= 3145728:
        report_list.append(filename)
    else:
        files_path_list.append(filename)
        
files_path_list = sorted(files_path_list)
# files_path_list

In [7]:
df_imgs = get_df_data(files_path_list)
df_imgs.head(20)

100%|██████████████████████████████████████████████████████████████████████████████████| 38/38 [00:09<00:00,  2.96it/s]


,[unknown],accession_number,actual_frame_duration,bits_allocated,bits_stored,cine_rate,columns,content_date,content_time,derivation_description,...,study_description,study_id,study_instance_uid,study_time,transducer_data_0,transducer_data_1,transducer_data_2,transducer_frequency,transducer_type,ultrasound_color_data_present
0,Phantom CHI OFF 4 MHz 10 cm C1-6,,NaN,8,8,NaN,264,20220531,112319,SmallPreview,...,ABD_PEDS,20220531.112241,1.2.840.113619.2.455.140911794567.1654021361.86,112241,C1-6,574,1373257431,4000,CURVED LINEAR,0
1,Phantom CHI OFF 4 MHz 15 cm C1-6,,NaN,8,8,NaN,264,20220531,112334,SmallPreview,...,ABD_PEDS,20220531.112241,1.2.840.113619.2.455.140911794567.1654021361.86,112241,C1-6,574,1373257431,4000,CURVED LINEAR,0
2,Phantom CHI OFF 4 MHz 20 cm C1-6,,NaN,8,8,NaN,264,20220531,112344,SmallPreview,...,ABD_PEDS,20220531.112241,1.2.840.113619.2.455.140911794567.1654021361.86,112241,C1-6,574,1373257431,4000,CURVED LINEAR,0
3,Phantom CHI ON 4.5 MHz 10 cm C1-6,,NaN,8,8,NaN,264,20220531,112350,SmallPreview,...,ABD_PEDS,20220531.112241,1.2.840.113619.2.455.140911794567.1654021361.86,112241,C1-6,574,1373257431,4500,CURVED LINEAR,0
4,Phantom CHI ON 4.5 MHz 15 cm C1-6,,NaN,8,8,NaN,264,20220531,112357,SmallPreview,...,ABD_PEDS,20220531.112241,1.2.840.113619.2.455.140911794567.1654021361.86,112241,C1-6,574,1373257431,4500,CURVED LINEAR,0
5,Phantom CHI ON 4.5 MHz 20 cm C1-6,,NaN,8,8,NaN,264,20220531,112407,SmallPreview,...,ABD_PEDS,20220531.112241,1.2.840.113619.2.455.140911794567.1654021361.86,112241,C1-6,574,1373257431,4500,CURVED LINEAR,0
6,Phantom UGAP Meas 1 & 2,,NaN,8,8,NaN,360,20220531,112421,SmallPreview,...,ABD_PEDS,20220531.112241,1.2.840.113619.2.455.140911794567.1654021361.86,112241,C1-6,574,1373257431,4500,CURVED LINEAR,0
7,Phantom UGAP Meas 2 & 4,,NaN,8,8,NaN,360,20220531,112440,SmallPreview,...,ABD_PEDS,20220531.112241,1.2.840.113619.2.455.140911794567.1654021361.86,112241,C1-6,574,1373257431,4500,CURVED LINEAR,0
8,Phantom UGAP Meas 5 & 6,,NaN,8,8,NaN,360,20220531,112459,SmallPreview,...,ABD_PEDS,20220531.112241,1.2.840.113619.2.455.140911794567.1654021361.86,112241,C1-6,574,1373257431,4500,CURVED LINEAR,0
9,Pat CHI On Long Liver-Kid - No Constraint,,NaN,8,8,NaN,264,20220531,112634,SmallPreview,...,ABD_PEDS,20220531.112241,1.2.840.113619.2.455.140911794567.1654021361.86,112241,C1-6,574,1373257431,4500,CURVED LINEAR,0


In [8]:
# get filename from glob.glob file paths
# dcmfiles = [os.path.basename(x) for x in files_path_list]
dcmfiles = list(map(os.path.basename, files_path_list))

In [9]:
dcmfiles, len(dcmfiles)

(['IM-0001-0400.dcm',
  'IM-0001-0600.dcm',
  'IM-0001-0800.dcm',
  'IM-0001-1000.dcm',
  'IM-0001-1200.dcm',
  'IM-0001-1400.dcm',
  'IM-0001-1600.dcm',
  'IM-0001-1800.dcm',
  'IM-0001-2000.dcm',
  'IM-0001-2400.dcm',
  'IM-0001-2600.dcm',
  'IM-0001-2800.dcm',
  'IM-0001-3000.dcm',
  'IM-0001-3200.dcm',
  'IM-0001-3400.dcm',
  'IM-0001-3600.dcm',
  'IM-0001-3800.dcm',
  'IM-0001-4000.dcm',
  'IM-0001-4200.dcm',
  'IM-0001-4400.dcm',
  'IM-0001-4600.dcm',
  'IM-0001-4800.dcm',
  'IM-0001-5000.dcm',
  'IM-0001-5200.dcm',
  'IM-0001-5400.dcm',
  'IM-0001-5600.dcm',
  'IM-0001-5610.dcm',
  'IM-0001-5800.dcm',
  'IM-0001-6000.dcm',
  'IM-0001-6200.dcm',
  'IM-0001-6400.dcm',
  'IM-0001-6600.dcm',
  'IM-0001-6800.dcm',
  'IM-0001-7000.dcm',
  'IM-0001-7200.dcm',
  'IM-0001-7400.dcm',
  'IM-0001-7719.dcm',
  'IM-0001-7894.dcm'],
 38)

In [10]:
df_imgs['dcmfile'] = dcmfiles
# df_imgs['jpgfile'] = jpgfiles     # Error: more jpg images than original dcm files 
# df_imgs['matfile'] = matfiles     # # Error: less mat files than original dcm files 

df_imgs
# df_imgs.shape

,[unknown],accession_number,actual_frame_duration,bits_allocated,bits_stored,cine_rate,columns,content_date,content_time,derivation_description,...,study_id,study_instance_uid,study_time,transducer_data_0,transducer_data_1,transducer_data_2,transducer_frequency,transducer_type,ultrasound_color_data_present,dcmfile
0,Phantom CHI OFF 4 MHz 10 cm C1-6,,NaN,8,8,NaN,264,20220531,112319,SmallPreview,...,20220531.112241,1.2.840.113619.2.455.140911794567.1654021361.86,112241,C1-6,574,1373257431,4000,CURVED LINEAR,0,IM-0001-0400.dcm
1,Phantom CHI OFF 4 MHz 15 cm C1-6,,NaN,8,8,NaN,264,20220531,112334,SmallPreview,...,20220531.112241,1.2.840.113619.2.455.140911794567.1654021361.86,112241,C1-6,574,1373257431,4000,CURVED LINEAR,0,IM-0001-0600.dcm
2,Phantom CHI OFF 4 MHz 20 cm C1-6,,NaN,8,8,NaN,264,20220531,112344,SmallPreview,...,20220531.112241,1.2.840.113619.2.455.140911794567.1654021361.86,112241,C1-6,574,1373257431,4000,CURVED LINEAR,0,IM-0001-0800.dcm
3,Phantom CHI ON 4.5 MHz 10 cm C1-6,,NaN,8,8,NaN,264,20220531,112350,SmallPreview,...,20220531.112241,1.2.840.113619.2.455.140911794567.1654021361.86,112241,C1-6,574,1373257431,4500,CURVED LINEAR,0,IM-0001-1000.dcm
4,Phantom CHI ON 4.5 MHz 15 cm C1-6,,NaN,8,8,NaN,264,20220531,112357,SmallPreview,...,20220531.112241,1.2.840.113619.2.455.140911794567.1654021361.86,112241,C1-6,574,1373257431,4500,CURVED LINEAR,0,IM-0001-1200.dcm
5,Phantom CHI ON 4.5 MHz 20 cm C1-6,,NaN,8,8,NaN,264,20220531,112407,SmallPreview,...,20220531.112241,1.2.840.113619.2.455.140911794567.1654021361.86,112241,C1-6,574,1373257431,4500,CURVED LINEAR,0,IM-0001-1400.dcm
6,Phantom UGAP Meas 1 & 2,,NaN,8,8,NaN,360,20220531,112421,SmallPreview,...,20220531.112241,1.2.840.113619.2.455.140911794567.1654021361.86,112241,C1-6,574,1373257431,4500,CURVED LINEAR,0,IM-0001-1600.dcm
7,Phantom UGAP Meas 2 & 4,,NaN,8,8,NaN,360,20220531,112440,SmallPreview,...,20220531.112241,1.2.840.113619.2.455.140911794567.1654021361.86,112241,C1-6,574,1373257431,4500,CURVED LINEAR,0,IM-0001-1800.dcm
8,Phantom UGAP Meas 5 & 6,,NaN,8,8,NaN,360,20220531,112459,SmallPreview,...,20220531.112241,1.2.840.113619.2.455.140911794567.1654021361.86,112241,C1-6,574,1373257431,4500,CURVED LINEAR,0,IM-0001-2000.dcm
9,Pat CHI On Long Liver-Kid - No Constraint,,NaN,8,8,NaN,264,20220531,112634,SmallPreview,...,20220531.112241,1.2.840.113619.2.455.140911794567.1654021361.86,112241,C1-6,574,1373257431,4500,CURVED LINEAR,0,IM-0001-2400.dcm


In [11]:
df_imgs.rename(columns={'[unknown]':'info'}, inplace=True)

# pd.set_option('display.max_columns', None)
# pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', -1)
df_imgs['info']

0     Phantom CHI OFF 4 MHz 10 cm C1-6                               
1     Phantom CHI OFF 4 MHz 15 cm C1-6                               
2     Phantom CHI OFF 4 MHz 20 cm C1-6                               
3     Phantom CHI ON 4.5 MHz 10 cm C1-6                              
4     Phantom CHI ON 4.5 MHz 15 cm C1-6                              
5     Phantom CHI ON 4.5 MHz 20 cm C1-6                              
6     Phantom UGAP Meas 1 & 2                                        
7     Phantom UGAP Meas 2 & 4                                        
8     Phantom UGAP Meas 5 & 6                                        
9     Pat CHI On Long Liver-Kid - No Constraint                      
10    Pat CHI On Long Liver Length - No Constraint                   
11    Pat CHI On Trans Bunny - No Constraint                         
12    Pat CHI On Trans Rt PV - No  Constraint                        
13    Pat CHI On Trans Rt Post PV - No Constraint                    
14    Pat CHI On Tra

In [12]:
# df_imgs.columns

In [13]:
# remove words
l = ['Choose Preset', 'Same Preset', 'New Breath Hold', 'Same Breath Hold', 'Probe', 'PRESS',
      '- No Constraint', 'Breath Hold', 'Lobe', 'Choose', 'new', ' Trans', 'C1-6', 'S,M,L', '- Measure ', 'Measure'
     'Preset 10,15,20', ' Measure L2-9 ', 'L2-9', 'RF CAPTURE ON ', ' Long', 'Trans', ' F6 -', 'No Constraint',
    'Breath ', 'Same as QUS', 'Same as', 'H\r\nold', 'No  Constraint', '-', 'Same Depth', ',', 'same', 'Same']

df_imgs['new_info'] = df_imgs['info']
results = df_imgs.new_info.str.replace('|'.join(l), '', regex=True).str.strip()

results

0     Phantom CHI OFF 4 MHz 10 cm       
1     Phantom CHI OFF 4 MHz 15 cm       
2     Phantom CHI OFF 4 MHz 20 cm       
3     Phantom CHI ON 4.5 MHz 10 cm      
4     Phantom CHI ON 4.5 MHz 15 cm      
5     Phantom CHI ON 4.5 MHz 20 cm      
6     Phantom UGAP Meas 1 & 2           
7     Phantom UGAP Meas 2 & 4           
8     Phantom UGAP Meas 5 & 6           
9     Pat CHI On LiverKid               
10    Pat CHI On Liver Length           
11    Pat CHI On Bunny                  
12    Pat CHI On Rt PV                  
13    Pat CHI On Rt Post PV             
14    Pat CHI On Rt Post PV             
15    IQ                                
16    Pat QUS#2 CHI Off Rt              
17    Pat QUS#3 CHI Off Rt              
18    Pat QUS#4 CHI Off Rt              
19    Pat QUS#5 CHI Off Rt              
20    Pat QUS#6 CHI On Rt               
21    Pat QUS#7 CHI On Rt               
22    Pat QUS#8 CHI On Rt               
23    Pat QUS#9 CHI On Rt               
24    Pat QUS#10

In [14]:
words = results.str.replace('  ', ' ', regex=True)
words = words.str.replace('  ', ' ', regex=True)
words = words.str.replace(' Meas ', '_', regex=True)
words = words.str.replace('Phant 19 ', 'Phant19_', regex=True)
words = words.str.replace('CHI On', 'ON', regex=True)
words = words.str.replace('CHI Off', 'OFF', regex=True)
words = words.str.replace('Liver-Kid', 'LiverKid', regex=True)
words = words.str.replace('Liv-Kid', 'LiverKid', regex=True)
words = words.str.replace('Liver Length', 'LiverLength', regex=True)
words = words.str.replace('Rf Post PV', 'RfPostPV', regex=True)
words = words.str.replace('Rt Post PV', 'RtPostPV', regex=True)
words = words.str.replace('Rt PV', 'RtPV', regex=True)
words = words.str.replace('IQ', 'QUS#1 OFF', regex=True)
words = words.str.replace('ABD Wall', 'ABDWall', regex=True)
words = words.str.replace('Cine Sweep', 'Cine', regex=True)
# words = words.str.replace('UGAP', 'Liver_UGAP', regex=True)
words = words.str.replace('PAT UGAP', 'UGAP_Liver', regex=True) #2129 Exam+1
words = words.str.replace('Phant UGAP', 'UGAP_Phant', regex=True)

words = words.str.replace('  ,', '', regex=True)
words = words.str.replace(' ,', '', regex=True)
words = words.str.replace(' & ', '_', regex=True)
words = words.str.replace('  ', ' ', regex=True)
words = words.str.replace('  ', ' ', regex=True)
words = words.str.replace(' ', '_', regex=True)
words = words.str.replace(',', '', regex=True)

words

0     Phantom_CHI_OFF_4_MHz_10_cm 
1     Phantom_CHI_OFF_4_MHz_15_cm 
2     Phantom_CHI_OFF_4_MHz_20_cm 
3     Phantom_CHI_ON_4.5_MHz_10_cm
4     Phantom_CHI_ON_4.5_MHz_15_cm
5     Phantom_CHI_ON_4.5_MHz_20_cm
6     Phantom_UGAP_1_2            
7     Phantom_UGAP_2_4            
8     Phantom_UGAP_5_6            
9     Pat_ON_LiverKid             
10    Pat_ON_LiverLength          
11    Pat_ON_Bunny                
12    Pat_ON_RtPV                 
13    Pat_ON_RtPostPV             
14    Pat_ON_RtPostPV             
15    QUS#1_OFF                   
16    Pat_QUS#2_OFF_Rt            
17    Pat_QUS#3_OFF_Rt            
18    Pat_QUS#4_OFF_Rt            
19    Pat_QUS#5_OFF_Rt            
20    Pat_QUS#6_ON_Rt             
21    Pat_QUS#7_ON_Rt             
22    Pat_QUS#8_ON_Rt             
23    Pat_QUS#9_ON_Rt             
24    Pat_QUS#10_ON_Rt            
25    Pat_ABDWall_ON_No_Contraint 
26    Pat_ABDWall_ON_No_Contraint 
27    Pat_UGAP_1_2                
28    Pat_UGAP_2_4  

In [15]:
df_imgs['new_name_info'] = words

In [16]:
df_imgs.columns

Index(['info', 'accession_number', 'actual_frame_duration', 'bits_allocated',
       'bits_stored', 'cine_rate', 'columns', 'content_date', 'content_time',
       'derivation_description', 'device_serial_number', 'effective_duration',
       'frame_delay', 'frame_increment_pointer', 'frame_time', 'heart_rate',
       'high_bit', 'image_type_0', 'image_type_1', 'image_type_2',
       'image_type_3', 'image_type_4', 'image_type_5', 'instance_number',
       'institution_name', 'lossy_image_compression',
       'lossy_image_compression_ratio', 'manufacturer',
       'manufacturer's_model_name', 'modality', 'number_of_frames',
       'operators'_name', 'patient's_age', 'patient's_birth_date',
       'patient's_birth_time', 'patient's_name', 'patient's_sex', 'patient_id',
       'patient_orientation', 'photometric_interpretation', 'physical_delta_x',
       'physical_delta_y', 'physical_units_x_direction',
       'physical_units_y_direction', 'pixel_representation',
       'planar_configura

In [17]:
new_names_df = df_imgs[["dcmfile", "patient_id", "content_date", "new_name_info"]]
new_names_df

,dcmfile,patient_id,content_date,new_name_info
0,IM-0001-0400.dcm,UP2127_US_2,20220531,Phantom_CHI_OFF_4_MHz_10_cm
1,IM-0001-0600.dcm,UP2127_US_2,20220531,Phantom_CHI_OFF_4_MHz_15_cm
2,IM-0001-0800.dcm,UP2127_US_2,20220531,Phantom_CHI_OFF_4_MHz_20_cm
3,IM-0001-1000.dcm,UP2127_US_2,20220531,Phantom_CHI_ON_4.5_MHz_10_cm
4,IM-0001-1200.dcm,UP2127_US_2,20220531,Phantom_CHI_ON_4.5_MHz_15_cm
5,IM-0001-1400.dcm,UP2127_US_2,20220531,Phantom_CHI_ON_4.5_MHz_20_cm
6,IM-0001-1600.dcm,UP2127_US_2,20220531,Phantom_UGAP_1_2
7,IM-0001-1800.dcm,UP2127_US_2,20220531,Phantom_UGAP_2_4
8,IM-0001-2000.dcm,UP2127_US_2,20220531,Phantom_UGAP_5_6
9,IM-0001-2400.dcm,UP2127_US_2,20220531,Pat_ON_LiverKid


In [18]:
ndf = new_names_df['patient_id'].str.split('US_',expand=True).rename(columns={0:"patientid", 1:"examid"})
af = pd.concat([new_names_df,ndf], 1)

af['examid'] = af['examid'].apply(lambda x: "{}{}".format('Exam', x))
af['patientid'] = af['patientid'].apply(lambda x: x.replace("_", ""))
af.drop("patient_id", axis=1, inplace= True)

In [19]:
af

,dcmfile,content_date,new_name_info,patientid,examid
0,IM-0001-0400.dcm,20220531,Phantom_CHI_OFF_4_MHz_10_cm,UP2127,Exam2
1,IM-0001-0600.dcm,20220531,Phantom_CHI_OFF_4_MHz_15_cm,UP2127,Exam2
2,IM-0001-0800.dcm,20220531,Phantom_CHI_OFF_4_MHz_20_cm,UP2127,Exam2
3,IM-0001-1000.dcm,20220531,Phantom_CHI_ON_4.5_MHz_10_cm,UP2127,Exam2
4,IM-0001-1200.dcm,20220531,Phantom_CHI_ON_4.5_MHz_15_cm,UP2127,Exam2
5,IM-0001-1400.dcm,20220531,Phantom_CHI_ON_4.5_MHz_20_cm,UP2127,Exam2
6,IM-0001-1600.dcm,20220531,Phantom_UGAP_1_2,UP2127,Exam2
7,IM-0001-1800.dcm,20220531,Phantom_UGAP_2_4,UP2127,Exam2
8,IM-0001-2000.dcm,20220531,Phantom_UGAP_5_6,UP2127,Exam2
9,IM-0001-2400.dcm,20220531,Pat_ON_LiverKid,UP2127,Exam2


In [20]:
#  content_date, new_name_info

In [21]:
af['new_filename'] = af["patientid"] + str("_") + af["examid"] + str("_") + af["content_date"] + str("_") + af['new_name_info'] + str("_") + af["dcmfile"]
af

,dcmfile,content_date,new_name_info,patientid,examid,new_filename
0,IM-0001-0400.dcm,20220531,Phantom_CHI_OFF_4_MHz_10_cm,UP2127,Exam2,UP2127_Exam2_20220531_Phantom_CHI_OFF_4_MHz_10_cm_IM-0001-0400.dcm
1,IM-0001-0600.dcm,20220531,Phantom_CHI_OFF_4_MHz_15_cm,UP2127,Exam2,UP2127_Exam2_20220531_Phantom_CHI_OFF_4_MHz_15_cm_IM-0001-0600.dcm
2,IM-0001-0800.dcm,20220531,Phantom_CHI_OFF_4_MHz_20_cm,UP2127,Exam2,UP2127_Exam2_20220531_Phantom_CHI_OFF_4_MHz_20_cm_IM-0001-0800.dcm
3,IM-0001-1000.dcm,20220531,Phantom_CHI_ON_4.5_MHz_10_cm,UP2127,Exam2,UP2127_Exam2_20220531_Phantom_CHI_ON_4.5_MHz_10_cm_IM-0001-1000.dcm
4,IM-0001-1200.dcm,20220531,Phantom_CHI_ON_4.5_MHz_15_cm,UP2127,Exam2,UP2127_Exam2_20220531_Phantom_CHI_ON_4.5_MHz_15_cm_IM-0001-1200.dcm
5,IM-0001-1400.dcm,20220531,Phantom_CHI_ON_4.5_MHz_20_cm,UP2127,Exam2,UP2127_Exam2_20220531_Phantom_CHI_ON_4.5_MHz_20_cm_IM-0001-1400.dcm
6,IM-0001-1600.dcm,20220531,Phantom_UGAP_1_2,UP2127,Exam2,UP2127_Exam2_20220531_Phantom_UGAP_1_2_IM-0001-1600.dcm
7,IM-0001-1800.dcm,20220531,Phantom_UGAP_2_4,UP2127,Exam2,UP2127_Exam2_20220531_Phantom_UGAP_2_4_IM-0001-1800.dcm
8,IM-0001-2000.dcm,20220531,Phantom_UGAP_5_6,UP2127,Exam2,UP2127_Exam2_20220531_Phantom_UGAP_5_6_IM-0001-2000.dcm
9,IM-0001-2400.dcm,20220531,Pat_ON_LiverKid,UP2127,Exam2,UP2127_Exam2_20220531_Pat_ON_LiverKid_IM-0001-2400.dcm


In [22]:
for old, new in zip(af['dcmfile'], af['new_filename']):
    os.rename(os.path.join(input_dir, old), os.path.join(input_dir, new))

______

In [23]:
file_names = os.listdir(input_dir)
for file_name in file_names:
    if 'frame' in file_name:
        os.remove(os.path.join(input_dir, file_name))
    elif 'Frame' in file_name:
        os.remove(os.path.join(input_dir, file_name))

_____

In [24]:
for old, new in zip(af['dcmfile'], af['new_filename']):
    old= old[:-4] + ".jpeg"
    new= new[:-4] + ".jpeg"
#     print(old, "==============", new)
    if os.path.exists(os.path.join(input_dir, old)):
        os.rename(os.path.join(input_dir, old), os.path.join(input_dir, new))

In [25]:
for old, new in zip(af['dcmfile'], af['new_filename']):
    old= old[:-4] + "_RF.mat"
    new= new[:-4] + "_RF.mat"
#     print(old, "==============", new)
    if os.path.exists(os.path.join(input_dir, old)):
        os.rename(os.path.join(input_dir, old), os.path.join(input_dir, new))

_______

In [26]:
files = af["new_filename"].to_list()
for filename in files:
    patient = filename[:6]
    exam = filename[7:12]

In [27]:
report_list_move = []
prefix = patient+ "_"+ exam + "_"
# prefix

for x in report_list:
    filename= os.path.basename(x)
#     print(filename)
    report_list_move.append(prefix + filename)

for old, new in zip(report_list, report_list_move):
#     print(old, "==============>", new)
    if os.path.exists(old):
        os.rename(old, os.path.join(input_dir, new))
        
for filename_move in report_list_move:
#     print(filename_move)
    shutil.move(os.path.join(input_dir, filename_move), human_dir)

______

In [28]:
patient_dir = os.path.join(sorted_dir, patient)
exam_dir = os.path.join(patient_dir, exam)

clinical_abd_wall = os.path.join(exam_dir, 'Clinical_Abd_Wall/')
clinical_cine = os.path.join(exam_dir, 'Clinical_Cine/')
clinical_static = os.path.join(exam_dir, 'Clinical_Static/')
# clinical_hri = os.path.join(exam_dir, 'Clinical_HRI/')

swe_liver = os.path.join(exam_dir, 'SWE_LIVER/')
ugap_liver = os.path.join(exam_dir, 'UGAP_LIVER/')
ugap_phantoms = os.path.join(exam_dir, 'UGAP_PHANTOMS/')

#######################
rf = os.path.join(exam_dir, 'RF/')
rf_harmonicsoff = os.path.join(rf, 'HarmonicsOFF/')
rf_harmonicson = os.path.join(rf, 'HarmonicsON/')

rf_liver_off = os.path.join(rf_harmonicsoff, 'Liver/')
rf_phantomcontemporaneous_off = os.path.join(rf_harmonicsoff, 'PhantomContemporaneous/')
rf_phantomstandard_off = os.path.join(rf_harmonicsoff, 'PhantomStandard/')

rf_liver_on = os.path.join(rf_harmonicson, 'Liver/')
rf_phantomcontemporaneous_on = os.path.join(rf_harmonicson, 'PhantomContemporaneous/')
rf_phantomstandard_on = os.path.join(rf_harmonicson, 'PhantomStandard/')

#######################
rf_sld = os.path.join(exam_dir, 'RF_With_SLD/')

rf_sld_harmonicsoff = os.path.join(rf_sld, 'HarmonicsOFF/')
rf_sld_harmonicson = os.path.join(rf_sld, 'HarmonicsON/')

rf_sld_liver_off = os.path.join(rf_sld_harmonicsoff, 'Liver/')
rf_sld_phantomcontemporaneous_off = os.path.join(rf_sld_harmonicsoff, 'PhantomContemporaneous/')
rf_sld_phantomstandard_off = os.path.join(rf_sld_harmonicsoff, 'PhantomStandard/')

rf_sld_liver_on = os.path.join(rf_sld_harmonicson, 'Liver/')
rf_sld_phantomcontemporaneous_on = os.path.join(rf_sld_harmonicson, 'PhantomContemporaneous/')
rf_sld_phantomstandard_on = os.path.join(rf_sld_harmonicson, 'PhantomStandard/')
#######################

if not os.path.exists(patient_dir):
    os.mkdir(patient_dir)

if not os.path.exists(exam_dir):
    os.mkdir(exam_dir)
    
    
if not os.path.exists(clinical_abd_wall):
    os.mkdir(clinical_abd_wall)

if not os.path.exists(clinical_cine):
    os.mkdir(clinical_cine)

if not os.path.exists(clinical_static):
    os.mkdir(clinical_static)
    
# if not os.path.exists(clinical_hri):
#     os.mkdir(clinical_hri)

if not os.path.exists(swe_liver):
    os.mkdir(swe_liver)

if not os.path.exists(ugap_liver):
    os.mkdir(ugap_liver)

if not os.path.exists(ugap_phantoms):
    os.mkdir(ugap_phantoms)

if not os.path.exists(rf):
    os.mkdir(rf)

if not os.path.exists(rf_sld):
    os.mkdir(rf_sld)
#######################
if not os.path.exists(rf):
    os.mkdir(rf)

if not os.path.exists(rf_harmonicsoff):
    os.mkdir(rf_harmonicsoff)

if not os.path.exists(rf_harmonicson):
    os.mkdir(rf_harmonicson)

if not os.path.exists(rf_liver_off):
    os.mkdir(rf_liver_off)

if not os.path.exists(rf_phantomcontemporaneous_off):
    os.mkdir(rf_phantomcontemporaneous_off)

if not os.path.exists(rf_phantomstandard_off):
    os.mkdir(rf_phantomstandard_off)

if not os.path.exists(rf_liver_on):
    os.mkdir(rf_liver_on)

if not os.path.exists(rf_phantomcontemporaneous_on):
    os.mkdir(rf_phantomcontemporaneous_on)

if not os.path.exists(rf_phantomstandard_on):
    os.mkdir(rf_phantomstandard_on)

if not os.path.exists(rf_sld):
    os.mkdir(rf_sld)

if not os.path.exists(rf_sld_harmonicsoff):
    os.mkdir(rf_sld_harmonicsoff)

if not os.path.exists(rf_sld_harmonicson):
    os.mkdir(rf_sld_harmonicson)

if not os.path.exists(rf_sld_liver_off):
    os.mkdir(rf_sld_liver_off)

if not os.path.exists(rf_sld_phantomcontemporaneous_off):
    os.mkdir(rf_sld_phantomcontemporaneous_off)

if not os.path.exists(rf_sld_phantomstandard_off):
    os.mkdir(rf_sld_phantomstandard_off)

if not os.path.exists(rf_sld_liver_on):
    os.mkdir(rf_sld_liver_on)

if not os.path.exists(rf_sld_phantomcontemporaneous_on):
    os.mkdir(rf_sld_phantomcontemporaneous_on)

if not os.path.exists(rf_sld_phantomstandard_on):
    os.mkdir(rf_sld_phantomstandard_on)

In [29]:
files = [f for f in os.listdir(input_dir) if os.path.isfile(os.path.join(input_dir, f))]
for filename in files:
    if "ABDWall" in filename:
        shutil.move(os.path.join(input_dir, filename), clinical_abd_wall)
        
    if "Cine" in filename:
        shutil.move(os.path.join(input_dir, filename), clinical_cine)
        
    if "Elasto" in filename:
        shutil.move(os.path.join(input_dir, filename), swe_liver)
      
    if "Bunny" in filename:
        shutil.move(os.path.join(input_dir, filename), clinical_static)
           
    if "LiverLength" in filename:
        shutil.copy(os.path.join(input_dir, filename), clinical_static)
        
    if "LiverKid" in filename:
        shutil.copy(os.path.join(input_dir, filename), clinical_static)
    
#     if "LiverLength" in filename:
#         shutil.move(os.path.join(input_dir, filename), clinical_static)
        
#     if "LiverKid" in filename:
#         shutil.move(os.path.join(input_dir, filename), clinical_static)
    
    if "RfPostPV" in filename:
        shutil.move(os.path.join(input_dir, filename), clinical_static)
        
    if "RtPostPV" in filename:
        shutil.move(os.path.join(input_dir, filename), clinical_static)
    
    if "RtPV" in filename:
        shutil.move(os.path.join(input_dir, filename), clinical_static)
    
    if "QUS" in filename:
        if "ON" in filename:
            shutil.move(os.path.join(input_dir, filename), rf_liver_on)
            
    if "QUS" in filename:
        if "OFF" in filename:
            shutil.move(os.path.join(input_dir, filename), rf_liver_off)
            
    if "UGAP_Phant" in filename:
        shutil.move(os.path.join(input_dir, filename), ugap_phantoms)
        
    if "UGAP_Phant" in filename:
        shutil.move(os.path.join(input_dir, filename), ugap_phantoms)        
    
    if "UGAP_Liver" in filename:
        shutil.move(os.path.join(input_dir, filename), ugap_liver)
        
    if "Phan" in filename:
        if "ON" in filename:
            shutil.move(os.path.join(input_dir, filename), rf_phantomcontemporaneous_on)
            
    if "Phan" in filename:
        if "OFF" in filename:
            shutil.move(os.path.join(input_dir, filename), rf_phantomcontemporaneous_on)
  
            
#     if "Phant" in filename:
#         if "ON" in filename:
#             shutil.move(os.path.join(input_dir, filename), rf_phantomstandard_on)
            
#     if "Phant" in filename:
#         if "OFF" in filename:
#             shutil.move(os.path.join(input_dir, filename), rf_phantomstandard_off)

_____

In [30]:
human_check(in_dir= input_dir)

In [31]:
empty_dirs_check("sorted_data/")

['sorted_data/UP2127\\Exam1\\Clinical_Abd_Wall',
 'sorted_data/UP2127\\Exam1\\RF\\HarmonicsOFF\\PhantomContemporaneous',
 'sorted_data/UP2127\\Exam1\\RF\\HarmonicsOFF\\PhantomStandard',
 'sorted_data/UP2127\\Exam1\\RF\\HarmonicsON\\PhantomStandard',
 'sorted_data/UP2127\\Exam1\\RF_With_SLD\\HarmonicsOFF\\Liver',
 'sorted_data/UP2127\\Exam1\\RF_With_SLD\\HarmonicsOFF\\PhantomContemporaneous',
 'sorted_data/UP2127\\Exam1\\RF_With_SLD\\HarmonicsOFF\\PhantomStandard',
 'sorted_data/UP2127\\Exam1\\RF_With_SLD\\HarmonicsON\\Liver',
 'sorted_data/UP2127\\Exam1\\RF_With_SLD\\HarmonicsON\\PhantomContemporaneous',
 'sorted_data/UP2127\\Exam1\\RF_With_SLD\\HarmonicsON\\PhantomStandard',
 'sorted_data/UP2127\\Exam1\\UGAP_LIVER',
 'sorted_data/UP2127\\Exam1\\UGAP_PHANTOMS',
 'sorted_data/UP2127\\Exam2\\RF\\HarmonicsOFF\\PhantomContemporaneous',
 'sorted_data/UP2127\\Exam2\\RF\\HarmonicsOFF\\PhantomStandard',
 'sorted_data/UP2127\\Exam2\\RF\\HarmonicsON\\PhantomStandard',
 'sorted_data/UP2127\\Exam2

_____